# DP Simulation of CIGRE MV with PV Inverters and Load Step

### Run simulation

In [ ]:
%%bash
TOP=${TOP:-$(git rev-parse --show-toplevel)}
PATH=${TOP}/build/Examples/Cxx

DP_CIGRE_MV_withDG_withLoadStep --timestep=1e-3 --duration=3 ${TOP}/build/_deps/cim-data-src/CIGRE_MV/NEPLAN/CIGRE_MV_no_tapchanger_noLoad1_LeftFeeder_With_LoadFlow_Results/*.xml

## Powerflow

### Read results

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re

# %matplotlib widget

modelName = 'DP_CIGRE_MV_withDG_withLoadStep_Powerflow'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

### Plot results

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name != 'N0.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

## Dynamic Phasor

In [ ]:
modelName = 'DP_CIGRE_MV_withDG_withLoadStep'
path = 'logs/' + modelName + '/'
dpsim_result_file = path + modelName + '.csv'

ts_dpsim = read_timeseries_csv(dpsim_result_file)

### Plot results

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name != 'N0.V' and ts_name[-2:] == '.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

## Voltages at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.V':
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### Plot zoomed to Load Step

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name != 'N0.V' and ts_name[-2:] == '.V':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
#plt.ylim([19000,21000])
plt.xlim([1.9,3])
plt.legend()
plt.show()

### DP backshift

In [ ]:
plt.figure(figsize=(12,12))
subplot_number = 1
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.V' and ts_name[:-2] != 'N0.V':
        plt.subplot(4,3,subplot_number)
        subplot_number += 1
        plt.plot(ts_dpsim[ts_name].interpolate(50e-6).time, ts_dpsim[ts_name].interpolate(50e-6).frequency_shift(50).values, label=ts_name)
        plt.legend(loc='upper right')
        plt.xlim([1.8,3])
        plt.ylim([18500,20000])
plt.show()

## Load currents

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[:4]=='LOAD':        
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
#plt.xlim([0,0.1])
plt.legend(loc='upper right')
plt.show()

## Line currents

In [ ]:
plt.figure(figsize=(12,6))
subplot_number = 1
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-2:] == '.I' and ts_name[:4] != 'LOAD':
        plt.subplot(4,3,subplot_number)
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
        subplot_number += 1
        plt.legend(loc='upper right')
#plt.xlim([0,0.1])
plt.show()

## PV inverter at N11

### States - Powers

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-7:]=='state_p' or ts_name[-7:]=='state_q':
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### States - Others

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if ts_name[-7:]!='state_p' and ts_name[-7:]!='state_q' and  'state' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### States at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'state' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### States at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'state' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))

### Plot inputs

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if 'input' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### Inputs at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'input' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### Inputs at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'input' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))

### Plot outputs

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if 'output' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

### Outputs at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'output' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))

### Outputs at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'output' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))

### Plot interface vars

In [ ]:
plt.figure(figsize=(12,6))
for ts_name, ts_obj  in ts_dpsim.items():
    if 'intf' in ts_name:
        plt.plot(ts_obj.time, ts_obj.abs().values, label=ts_name)
plt.legend()
plt.show()

## Plot derived power

In [ ]:
pv_N11_s_intf = ts_dpsim['pv_N11_v_intf'].values*np.conj(ts_dpsim['pv_N11_i_intf'].values)
plt.figure(figsize=(12,6))
plt.plot(ts_dpsim['pv_N11_v_intf'].time, np.real(pv_N11_s_intf), label='pv_N11_P_intf')
plt.plot(ts_dpsim['pv_N11_v_intf'].time, np.imag(pv_N11_s_intf), label='pv_N11_Q_intf')
plt.legend()
plt.show()


### Interface vars at first timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'intf' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[0]) + ', ' + str(ts_obj.phase().values[0]))
print('pv_N11_s_intf' + ': ' + str(np.real(pv_N11_s_intf[0])) + ', ' + str(np.imag(pv_N11_s_intf[0])))

### Interface vars at last timestep

In [ ]:
for ts_name, ts_obj  in ts_dpsim.items():
    if 'intf' in ts_name:
        print(ts_name + ': ' + str(ts_obj.abs().values[-1]) + ', ' + str(ts_obj.phase().values[-1]))
print('pv_N11_s_intf' + ': ' + str(np.real(pv_N11_s_intf[-1])) + ', ' + str(np.imag(pv_N11_s_intf[-1])))